## Pandas Assignment Part 2
**Melt, Pivot, Aggregation, Iteration, Groupby**

In [1]:
import pandas as pd

In [28]:
# Loading in Diabetes Dataset #
raw_data = pd.read_csv("diabetic_data.csv", header = 0)
print(raw_data.shape)
raw_data.head()

(101766, 50)


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


#### GroupBy
Group DataFrame using a mapper or by a Series of columns.

#### Aggregate
Perform one or more operations on data based on specified axis

In [60]:
# Compare the time spent in hospital between genders
# Check the values within the raw_data gender column
print(raw_data['gender'].value_counts())

# Filter for Male and Female and check for their removal in filt_data
filt_data = raw_data[raw_data['gender'].isin(["Male", "Female"])]
print(filt_data['gender'].value_counts())

# Group the data by the gender value and find the mean time spent in hospital
gender_mean = filt_data.groupby("gender").time_in_hospital.agg("mean")
gender_mean

gender
Female             54708
Male               47055
Unknown/Invalid        3
Name: count, dtype: int64
gender
Female    54708
Male      47055
Name: count, dtype: int64


gender
Female    4.478797
Male      4.299777
Name: time_in_hospital, dtype: float64

#### Pivot 
Produce a pivot table based on column values
- take one column and transition it into multiple

In [87]:
# Pivot Table
# Determine average time spent in hospital for various admission types
# Grab necessary columns
pivot_table = raw_data[['encounter_id', "patient_nbr","admission_type_id", "time_in_hospital"]]
# Create a pivot table
# Index will be the encounter ID with the columns being the admission type ID and values being the time spent in hospital
# Take the mean of each column
pivot_table = pivot_table.pivot(index = "encounter_id", columns = "admission_type_id", values = "time_in_hospital")

pivot_table.mean()



admission_type_id
1    4.378274
2    4.610714
3    4.320473
4    3.200000
5    3.947126
6    4.583066
7    4.857143
8    3.062500
dtype: float64

#### Melt
Transition a data frame from wide to long format
- One column contains observation identifiers
- The remaining columns contain measured observations


In [112]:
# Reshaping the patient number and insulin column
pre_melt = raw_data[['patient_nbr', 'insulin']]
# the patient number is the id variable
post_melt = pre_melt.melt(id_vars = "patient_nbr", value_vars="insulin", value_name="insulin_status")
post_melt


,patient_nbr,variable,insulin_status
0,8222157,insulin,No
1,55629189,insulin,Up
2,86047875,insulin,No
3,82442376,insulin,Up
4,42519267,insulin,Steady
...,...,...,...
101761,100162476,insulin,Down
101762,74694222,insulin,Steady
101763,41088789,insulin,Down
101764,31693671,insulin,Up


#### Iter
iterrows() returns the iterator yielding each index value along with a series containing the data in each row.

In [126]:
# Creating a dictionary of insulin values for each patient
# key = patient number, value = list of insulin values

insulin_tracker = {}

for row_index,row in post_melt.iterrows():
    if row[0] in insulin_tracker.keys():
      insulin_tracker[row[0]].append(row[2])
    else:
      insulin_tracker[row[0]] = [row[2]]

insulin_tracker

/tmp/ipykernel_1113/3799072883.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] in insulin_tracker.keys():
/tmp/ipykernel_1113/3799072883.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  insulin_tracker[row[0]] = [row[2]]
/tmp/ipykernel_1113/3799072883.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  insulin_tracker[row[0]].append(row[2])


{8222157: ['No'],
 55629189: ['Up'],
 86047875: ['No'],
 82442376: ['Up'],
 42519267: ['Steady'],
 82637451: ['Steady'],
 84259809: ['Steady'],
 114882984: ['No'],
 48330783: ['Steady'],
 63555939: ['Steady'],
 89869032: ['Steady'],
 77391171: ['Steady'],
 85504905: ['Down', 'Steady'],
 77586282: ['Steady'],
 49726791: ['Steady'],
 86328819: ['Up'],
 92519352: ['Steady'],
 108662661: ['No'],
 107389323: ['Steady'],
 69422211: ['Steady'],
 22864131: ['Down'],
 21239181: ['Steady'],
 63000108: ['No'],
 107400762: ['No'],
 62718876: ['Steady'],
 21861756: ['No'],
 40523301: ['No'],
 115196778: ['Steady', 'Down', 'Steady', 'Steady', 'Steady'],
 41606064: ['Down', 'Steady', 'Steady'],
 18196434: ['No'],
 56480238: ['Steady'],
 96664626: ['Down'],
 80845353: ['Steady', 'Down', 'No'],
 114715242: ['Steady', 'Down'],
 3327282: ['Down'],
 63023292: ['Down', 'Steady', 'Steady'],
 98427861: ['Steady'],
 112002975: ['Steady', 'Steady'],
 101002446: ['Steady'],
 104672268: ['Steady'],
 80588529: ['